# CrewAI Multi‑Agent Graded Mini Project
## UseCase : Employee Support & Policy Guidance (Non‑RAG, rule/policy‑driven)

This notebook implements a goal‑oriented **multi‑agent workflow** using **CrewAI** for HR Operations.
It follows the Week 11 architectural pattern (identify → reason with policy → draft response → escalate decision),
but in the **HR domain**.


## 0) Setup & Notes
**You need:** Python 3.10/3.11/3.12, `crewai`, and an OpenAI API key in environment variable `OPENAI_API_KEY`.

**Why non‑RAG?** Policies are simulated via static text + hardcoded rules.

**Safety:** Some HR queries are sensitive (harassment, discrimination, medical, payroll identity) and should escalate.


In [ ]:

import os

# Option 1 (recommended): set OPENAI_API_KEY in your OS environment variables.
# Option 2: set it here (not recommended for submissions):
# os.environ["OPENAI_API_KEY"] = "YOUR_KEY_HERE"

print("OPENAI_API_KEY set:", bool(os.environ.get("OPENAI_API_KEY")))


## 1) Install (only if needed)
If running locally and packages are missing, run:

```bash
pip install -U crewai langchain-openai pydantic python-dotenv ipykernel jupyter
```

(You can skip in Vocareum if already available.)

## 2) Imports & Data Models
We use a strict `Handoff` model to store outputs and ensure predictable flow.

In [ ]:

import json
import re
from typing import Any, Dict, List, Optional

from pydantic import BaseModel, Field
from crewai import Agent, Task, Crew


In [ ]:

class Handoff(BaseModel):
    employee_query: str
    category: Optional[str] = None
    confidence: Optional[float] = None

    applicable_policies: List[str] = Field(default_factory=list)
    policy_notes: Optional[str] = None

    draft_response: Optional[str] = None

    escalate: bool = False
    escalation_reason: Optional[str] = None
    risk_score: Optional[int] = None


## 3) Static HR Policy Context (Non‑RAG)
These are simplified policies used by the agents as **static knowledge**.

In [ ]:

HR_POLICIES = {
    "leave_policy": {
        "summary": (
            "Annual leave: employees may apply via HR portal. "
            "Sick leave may require medical certificate beyond a threshold. "
            "Leave balance visible in HR portal."
        ),
        "allowed_actions": [
            "explain leave types and how to apply",
            "share where to check leave balance",
            "advise required documents for sick leave (if applicable)",
        ],
        "escalate_when": [
            "medical details shared",
            "manager dispute about leave approval",
        ],
    },
    "payroll_policy": {
        "summary": (
            "Payroll runs monthly. Payslips are available in HR portal. "
            "Bank account changes require secure verification. "
            "Do not ask for sensitive info (full bank details, OTP)."
        ),
        "allowed_actions": [
            "explain payslip access",
            "outline payroll correction process",
            "advise contacting payroll desk for discrepancies",
        ],
        "escalate_when": [
            "identity verification required",
            "salary not credited",
            "bank account change request",
        ],
    },
    "benefits_policy": {
        "summary": (
            "Benefits include health insurance enrollment, dependents management, "
            "and reimbursements (per company rules)."
        ),
        "allowed_actions": [
            "explain enrollment steps",
            "share where to update dependents",
            "explain reimbursement claim steps",
        ],
        "escalate_when": [
            "claim rejection dispute",
            "medical/diagnosis details",
        ],
    },
    "transfer_policy": {
        "summary": (
            "Internal transfers require current manager awareness and role availability. "
            "Process is via internal job portal."
        ),
        "allowed_actions": [
            "explain internal transfer process",
            "share internal job portal steps",
            "advise discussing with manager",
        ],
        "escalate_when": [
            "conflict with manager",
            "urgent relocation due to personal reasons (sensitive)",
        ],
    },
    "conduct_policy": {
        "summary": (
            "Harassment, discrimination, retaliation, threats, and severe grievances "
            "must be escalated to HR/Compliance immediately. Confidential handling applies."
        ),
        "allowed_actions": [
            "acknowledge concern and advise official reporting channel",
            "provide high-level next steps and confidentiality note",
        ],
        "escalate_when": [
            "harassment/discrimination",
            "threats/violence",
            "legal claims",
        ],
    },
}


## 4) Helper: JSON parsing & deterministic risk scoring
CrewAI agents sometimes include extra text; we extract the JSON block safely.
Risk score is a static heuristic for escalation safeguards.

In [ ]:

def parse_json_strict(text: str) -> Dict[str, Any]:
    text = str(text).strip()
    try:
        return json.loads(text)
    except Exception:
        # Grab the first JSON object found
        match = re.search(r"\{[\s\S]*\}", text)
        if not match:
            raise ValueError(f"No JSON object found. Raw output (first 500 chars):\n{text[:500]}")
        return json.loads(match.group(0))


SENSITIVE_KEYWORDS = [
    "harassment", "sexual", "discrimination", "abuse", "threat", "violence", "retaliation",
    "legal", "lawsuit", "court", "police",
    "medical", "diagnosis", "pregnant", "mental", "depression",
    "otp", "password", "pan", "aadhaar", "passport", "bank account number",
]

HIGH_URGENCY = ["urgent", "immediately", "asap", "today", "right now"]

def simple_risk_score(query: str, category: Optional[str], confidence: Optional[float]) -> int:
    q = (query or "").lower()
    score = 0

    # Sensitivity / compliance
    if any(k in q for k in SENSITIVE_KEYWORDS):
        score += 45

    # Categories that often need escalation
    if category in {"conduct_grievance"}:
        score += 45
    if category in {"payroll_issue"} and ("not credited" in q or "salary not received" in q):
        score += 25

    # Urgency signals
    if any(k in q for k in HIGH_URGENCY):
        score += 10

    # Low confidence -> risk
    if confidence is not None and confidence < 0.6:
        score += 20

    return min(score, 100)


## 5) Agent Prompts
Each agent has a tight role with structured JSON output to enable clean handoffs.

In [ ]:

INTENT_AGENT_PROMPT = """You are an HR Query Classification Agent.
Classify the employee query into ONE of:
- leave_request
- payroll_issue
- benefits_query
- internal_transfer
- conduct_grievance
- general_hr

Return ONLY valid JSON with keys:
category (string), confidence (number 0 to 1)

Employee query:
{employee_query}
"""

POLICY_AGENT_PROMPT = """You are an HR Policy Reasoning Agent.
You receive an employee query and its category. Use ONLY the static HR_POLICIES content provided below.
Decide which policy areas apply and what actions are allowed.

HR_POLICIES (static):
{policies_text}

Return ONLY valid JSON with keys:
applicable_policies (array of policy keys),
allowed_actions (array of strings),
policy_notes (string)

Employee query: {employee_query}
Category: {category}
Confidence: {confidence}
"""

RESPONSE_AGENT_PROMPT = """You are an HR Response Drafting Agent.
Draft a helpful, professional response following the allowed actions and policy notes.
Never request sensitive information (OTP, passwords, full IDs, full bank details).
If the query is sensitive, keep the response supportive and point to official channels.

Return ONLY valid JSON with key:
draft_response (string)

Employee query: {employee_query}
Category: {category}
Allowed actions: {allowed_actions}
Policy notes: {policy_notes}
"""

ESCALATION_AGENT_PROMPT = """You are an HR Escalation Decision Agent.
Decide whether a human HR rep must handle this.
Escalate if:
- conduct/grievance topics (harassment/discrimination/threats/legal)
- identity verification or bank/payslip security is required
- low confidence or unclear details
- any sensitive personal/medical information appears

Return ONLY valid JSON with keys:
escalate (true/false), escalation_reason (string)

Employee query: {employee_query}
Category: {category}
Confidence: {confidence}
Policy notes: {policy_notes}
Draft response: {draft_response}
"""


## 6) Define Agents
We create 4 agents with clear roles and goals.

In [ ]:

# NOTE: CrewAI will use your configured LLM provider (via environment variables).
# In many setups, CrewAI uses OpenAI under the hood when OPENAI_API_KEY is set.

intent_agent = Agent(
    role="HR Intent Classification Agent",
    goal="Classify the employee query into the correct HR category with a confidence score.",
    backstory="You are an HR triage assistant. You are precise and return strict JSON only.",
    verbose=False,
)

policy_agent = Agent(
    role="HR Policy Reasoning Agent",
    goal="Select applicable HR policies and allowed actions using static policy text only.",
    backstory="You are an HR policy specialist. You do not invent policies outside the provided text.",
    verbose=False,
)

response_agent = Agent(
    role="HR Response Drafting Agent",
    goal="Draft a safe, helpful response consistent with policies and allowed actions.",
    backstory="You write clear HR responses and avoid collecting sensitive data.",
    verbose=False,
)

escalation_agent = Agent(
    role="HR Escalation Decision Agent",
    goal="Decide whether the case must be escalated to a human HR rep and explain why.",
    backstory="You prioritize confidentiality, compliance, and safety.",
    verbose=False,
)


## 7) Workflow Orchestrator (Sequential Handoffs)
We run the agents sequentially so each depends on prior outputs.
To avoid “running forever”, we:
- keep prompts concise
- parse strict JSON
- apply deterministic escalation safeguards


In [ ]:

def run_hr_workflow(employee_query: str, verbose: bool = False) -> Handoff:
    policies_text = json.dumps(
        {k: {"summary": v["summary"], "allowed_actions": v["allowed_actions"], "escalate_when": v["escalate_when"]}
         for k, v in HR_POLICIES.items()},
        indent=2
    )

    # --- 1) Intent ---
    intent_task = Task(
        description=INTENT_AGENT_PROMPT.format(employee_query=employee_query),
        expected_output="JSON with keys: category, confidence",
        agent=intent_agent,
    )
    intent_crew = Crew(agents=[intent_agent], tasks=[intent_task], verbose=verbose)
    intent_raw = intent_crew.kickoff()
    intent_out = parse_json_strict(intent_raw)

    category = intent_out.get("category")
    confidence = intent_out.get("confidence")

    # --- 2) Policy reasoning ---
    policy_task = Task(
        description=POLICY_AGENT_PROMPT.format(
            policies_text=policies_text,
            employee_query=employee_query,
            category=category,
            confidence=confidence,
        ),
        expected_output="JSON with keys: applicable_policies, allowed_actions, policy_notes",
        agent=policy_agent,
    )
    policy_crew = Crew(agents=[policy_agent], tasks=[policy_task], verbose=verbose)
    policy_raw = policy_crew.kickoff()
    policy_out = parse_json_strict(policy_raw)

    allowed_actions = policy_out.get("allowed_actions", [])
    policy_notes = policy_out.get("policy_notes")

    # --- 3) Response drafting ---
    response_task = Task(
        description=RESPONSE_AGENT_PROMPT.format(
            employee_query=employee_query,
            category=category,
            allowed_actions=allowed_actions,
            policy_notes=policy_notes,
        ),
        expected_output="JSON with key: draft_response",
        agent=response_agent,
    )
    response_crew = Crew(agents=[response_agent], tasks=[response_task], verbose=verbose)
    response_raw = response_crew.kickoff()
    response_out = parse_json_strict(response_raw)

    draft_response = response_out.get("draft_response")

    # --- 4) Escalation decision ---
    escalation_task = Task(
        description=ESCALATION_AGENT_PROMPT.format(
            employee_query=employee_query,
            category=category,
            confidence=confidence,
            policy_notes=policy_notes,
            draft_response=draft_response,
        ),
        expected_output="JSON with keys: escalate, escalation_reason",
        agent=escalation_agent,
    )
    escalation_crew = Crew(agents=[escalation_agent], tasks=[escalation_task], verbose=verbose)
    escalation_raw = escalation_crew.kickoff()
    escalation_out = parse_json_strict(escalation_raw)

    handoff = Handoff(
        employee_query=employee_query,
        category=category,
        confidence=confidence,
        applicable_policies=policy_out.get("applicable_policies", []),
        policy_notes=policy_notes,
        draft_response=draft_response,
        escalate=bool(escalation_out.get("escalate")),
        escalation_reason=escalation_out.get("escalation_reason") or "",
    )

    # Deterministic risk safeguard
    handoff.risk_score = simple_risk_score(employee_query, handoff.category, handoff.confidence)
    if handoff.risk_score is not None and handoff.risk_score >= 70:
        handoff.escalate = True
        if not handoff.escalation_reason:
            handoff.escalation_reason = "High risk score based on static rules."

    return handoff


## 8) Quick Test (Single Query)
Run one query first to confirm everything works.

In [ ]:

q = "How do I apply for annual leave and check my leave balance?"
out = run_hr_workflow(q, verbose=False)
print(out.model_dump_json(indent=2))


## 9) Full Test Set (includes escalation edge cases)
These cover routine + sensitive cases.

In [ ]:

TEST_QUERIES = [
    "How do I apply for annual leave and check my leave balance?",
    "My payslip amount looks wrong this month. What should I do?",
    "How do I add my spouse as a dependent in the health insurance?",
    "How can I apply for an internal transfer to another team?",
    # Edge / sensitive
    "I am being harassed by a colleague and I feel unsafe. Urgent.",
    "My salary was not credited and I need it today.",
    "Can you verify my identity using OTP and update my bank account details?",
    "I want to share medical diagnosis details for sick leave approval.",
]

for q in TEST_QUERIES:
    print("\n" + "="*100)
    print("EMPLOYEE QUERY:", q)
    out = run_hr_workflow(q, verbose=False)
    print(out.model_dump_json(indent=2))


## 10) Architecture Summary (1–2 pages content)
Copy this section into your PDF write‑up if needed.

### Agents
1. **HR Intent Classification Agent** — Categorizes query (leave/payroll/benefits/transfer/conduct/general) with confidence.
2. **HR Policy Reasoning Agent** — Uses static HR_POLICIES to pick applicable policy areas and allowed actions.
3. **HR Response Drafting Agent** — Produces a safe response following allowed actions; avoids collecting sensitive info.
4. **HR Escalation Decision Agent** — Decides escalation based on sensitivity, security, low confidence, or policy triggers.

### Task Flow / Handoffs
Sequential: **Intent → Policy → Response → Escalation**. Downstream agents explicitly consume upstream outputs.

### Escalation Logic
- LLM‑based escalation agent decision + **deterministic risk safeguard** (keywords, category, confidence).
- Automatically escalates for conduct/grievance and identity/security flows.

### Design Rationale
- Non‑RAG compliance: policies are static context; no external retrieval.
- Strict JSON: simplifies orchestration and grading reproducibility.
- Safety: avoids collecting secrets and escalates sensitive HR cases.
